In [2]:
import requests
import time

class LegalMixtralAPIAnswerer:
    def __init__(self, hf_token: str, model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1", max_new_tokens: int = 300):
        self.api_url = f"https://api-inference.huggingface.co/models/{model_name}"
        self.headers = {
            "Authorization": f"Bearer {hf_token}",
            "Content-Type": "application/json"
        }
        self.max_new_tokens = max_new_tokens

    def build_prompt(self, query: str, context_docs: list) -> str:
        context = "\n\n".join([f"Context {i+1}:\n{doc.strip()}" for i, doc in enumerate(context_docs)])
        prompt = (
            f"{context}\n\n"
            f"Question: {query.strip()}\n\n"
            f"Instructions: Based only on the above legal context, first provide a short answer to the question, "
            f"then explain your reasoning in a separate paragraph.\n\n"
            f"Answer:"
        )
        return prompt

    def generate(self, query: str, context_docs: list, debug: bool = False) -> dict:
        prompt = self.build_prompt(query, context_docs)
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": self.max_new_tokens,
                "do_sample": False,
                "temperature": 0.2,
                "return_full_text": False
            }
        }

        if debug:
            print("🚀 Prompt sent to Hugging Face:\n", prompt)
            print("-" * 80)

        response = requests.post(self.api_url, headers=self.headers, json=payload)

        if response.status_code == 503:
            estimated_time = response.json().get("estimated_time", 10)
            print(f"⏳ Model loading... waiting {estimated_time} seconds")
            time.sleep(estimated_time)
            return self.generate(query, context_docs, debug)

        if response.status_code != 200:
            raise RuntimeError(f"❌ HF API Error {response.status_code}: {response.text}")

        full_output = response.json()[0]["generated_text"].strip()

        # Try to split answer vs reasoning
        if "Reasoning:" in full_output:
            answer_part, reasoning_part = full_output.split("Reasoning:", 1)
        else:
            lines = full_output.split("\n", 1)
            answer_part = lines[0]
            reasoning_part = lines[1] if len(lines) > 1 else ""

        return {
            "answer": answer_part.strip(),
            "reasoning": reasoning_part.strip()
        }

In [4]:
hf_token = "use_hf_token_here"
query = "Under what conditions can evidence from a warrantless search be admissible?"
context_docs = [
    "The Fourth Amendment guarantees protection from unreasonable searches.",
    "Evidence obtained without a warrant is usually inadmissible, but there are exceptions.",
    "These include exigent circumstances, consent, and search incident to lawful arrest.",
    "In Smith v. Ohio, the Court ruled that reasonable suspicion can sometimes justify limited searches."
]

llm = LegalMixtralAPIAnswerer(hf_token)
response = llm.generate(query, context_docs, debug=True)

print("\n🧠 Answer:", response["answer"])
print("📚 Reasoning:", response["reasoning"])

🚀 Prompt sent to Hugging Face:
 Context 1:
The Fourth Amendment guarantees protection from unreasonable searches.

Context 2:
Evidence obtained without a warrant is usually inadmissible, but there are exceptions.

Context 3:
These include exigent circumstances, consent, and search incident to lawful arrest.

Context 4:
In Smith v. Ohio, the Court ruled that reasonable suspicion can sometimes justify limited searches.

Question: Under what conditions can evidence from a warrantless search be admissible?

Instructions: Based only on the above legal context, first provide a short answer to the question, then explain your reasoning in a separate paragraph.

Answer:
--------------------------------------------------------------------------------

🧠 Answer: Evidence from a warrantless search can be admissible under the following conditions: exigent circumstances, consent, search incident to lawful arrest, and when there is reasonable suspicion.
📚 Reasoning: Explanation: The Fourth Amendment 